In [1]:
import os
from datetime import datetime
import shutil
import yaml
from tqdm import tqdm
import torch
import numpy as np
import sys
import sys
import matplotlib as mpl
%matplotlib inline 
import scipy.io as sio
import logging
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from collections import namedtuple, defaultdict

In [2]:
save_path="/Users/eghbalhosseini/MyData/semantic_proj/"
model_type="gpt2-xl"
LAYER_COUNT = 48
FEATURE_COUNT = 1024
access_rights = 0o755
try:
    os.mkdir(save_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_path)
model_path=os.path.join(save_path,model_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)

Creation of /Users/eghbalhosseini/MyData/semantic_proj/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/gpt2-xl failed



In [3]:
# configure the gpt2 model 
config=GPT2Config.from_pretrained(model_type)
config.output_hidden_states=True
# make model from config
model=GPT2Model(config)
model.from_pretrained(model_type)
device = torch.device("cpu")
model.to(device)
# 
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

KeyboardInterrupt: 

In [ ]:
def match_tokenized_to_untokenized_gpt2(tokenized_sent, untokenized_sent):
    mapping = defaultdict(list)
    untokenized_sent_index = 0
    tokenized_sent_index = 0
    while (untokenized_sent_index < len(untokenized_sent) and
        tokenized_sent_index < len(tokenized_sent)):
      while (tokenized_sent_index + 1 < len(tokenized_sent) and
          not(tokenized_sent[tokenized_sent_index + 1].startswith("Ġ"))):
        mapping[untokenized_sent_index].append(tokenized_sent_index)
        tokenized_sent_index += 1
      mapping[untokenized_sent_index].append(tokenized_sent_index)
      untokenized_sent_index += 1
      tokenized_sent_index += 1
    return mapping

# create a few example sentences 

In [5]:
lineList=['tiger is more dangerous than dolphin',
          'elephant is bigger than tiger',
         'chicken is smaller than elephant',
         'tiger faster than elephant',
         'turtle is slower than dolphin',
         'elephant is heavier than tiger']

In [6]:
representation_arr = np.zeros((len(lineList),2), dtype=np.object)

In [9]:
# get sentence lines 
for index, line in tqdm(enumerate(lineList)):
    untokenized_sent = line.strip().split()
    tokenized_sent = tokenizer.tokenize(line,add_prefix_space=True)
    untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_sent, untokenized_sent)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sent)
    segment_ids = [1 for x in tokenized_sent]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    with torch.no_grad():
        encoded_layers = model(tokens_tensor)
        last_hidden_state=encoded_layers[0]
        hidden_output=encoded_layers[2]
        layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_sent))], dim=0) for F in hidden_output]
        layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
        rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
        representation_arr[index,0]=' '.join(untokenized_sent)
        representation_arr[index,1]=rep_arr
sio.savemat(os.path.join(model_path, 'sentence_representation_cells_all_layers.mat'), {'sentence_representation_arr':representation_arr})

6it [00:02,  2.54it/s]
